QUESTION 1

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=2718356e23e88fc19be4b00e5c55d0d85adcb0d34dd1c553bb56dd1eebf74e43
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from pyspark import SparkContext, SparkConf
from collections import Counter
import itertools
from pyspark.sql import *
from pyspark.sql.functions import *
#To create the Spark Session
spark = SparkSession.builder.getOrCreate()

#To create the Spark Context
sc = spark.sparkContext

In [5]:
fileName = 'soc-LiveJournal1Adj.txt'
#Setting the number of friends to be recommended as 10
N = 10  

def search_connections(line):
  default_minimum = -9999999999
  user, friends = line.split('\t')
  friends = friends.split(',')
  mutual_friend = [(pair, 1) for pair in itertools.permutations(friends, 2)]
  connections = [((user, friend), default_minimum) for friend in friends]
  return connections + mutual_friend


recommendations = (sc.textFile( fileName, 16 ).flatMap( search_connections ).reduceByKey( lambda total, current: total + current ).filter(lambda num_of_pairs: num_of_pairs[1] > 0).map(lambda num_of_user_friends: (num_of_user_friends[0][0], (num_of_user_friends[1], num_of_user_friends[0][1]))).groupByKey().map(lambda suggestions:(suggestions[0], Counter( dict( (friend, num) for num, friend in suggestions[1] ) ).most_common( N ) ) )
                   )

In [6]:
print("Friends recommended for userID 924:")
print(recommendations.lookup('924'))
print("Friends recommended for userID 8941:")
print(recommendations.lookup('8941'))
print("Friends recommended for userID 8942:")
print(recommendations.lookup('8942'))
print("Friends recommended for userID 9019:")
print(recommendations.lookup('9019'))
print("Friends recommended for userID 9020:")
print(recommendations.lookup('9020'))
print("Friends recommended for userID 9021:")
print(recommendations.lookup('9021'))
print("Friends recommended for userID 9022:")
print(recommendations.lookup('9022'))
print("Friends recommended for userID 9990:")
print(recommendations.lookup('9990'))
print("Friends recommended for userID 9992:")
print(recommendations.lookup('9992'))
print("Friends recommended for userID 9993:")
print(recommendations.lookup('9993'))

Friends recommended for userID 924:
[[('43748', 1), ('15416', 1), ('45881', 1), ('6995', 1), ('11860', 1), ('439', 1), ('2409', 1)]]
Friends recommended for userID 8941:
[[('8944', 2), ('8943', 2), ('8940', 1)]]
Friends recommended for userID 8942:
[[('8939', 3), ('8944', 1), ('8940', 1), ('8943', 1)]]
Friends recommended for userID 9019:
[[('9022', 2), ('317', 1), ('9023', 1)]]
Friends recommended for userID 9020:
[[('9021', 3), ('9017', 2), ('9022', 2), ('9016', 2), ('317', 1), ('9023', 1)]]
Friends recommended for userID 9021:
[[('9020', 3), ('9016', 2), ('9022', 2), ('9017', 2), ('9023', 1), ('317', 1)]]
Friends recommended for userID 9022:
[[('9020', 2), ('9021', 2), ('9019', 2), ('9017', 1), ('9016', 1), ('317', 1), ('9023', 1)]]
Friends recommended for userID 9990:
[[('13478', 1), ('34485', 1), ('34299', 1), ('34642', 1), ('13877', 1), ('37941', 1), ('13134', 1)]]
Friends recommended for userID 9992:
[[('9987', 4), ('9989', 4), ('35667', 3), ('9991', 2)]]
Friends recommended for